# EarthQauke Predictor

In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data = pd.read_csv("data/earthquake_data.csv")

In [3]:
data.head()

,Date,Time,Latitude,Longitude,Type,Depth,Magnitude,NormalizedLat,NormalizedLong,NormalizedDepth,NormalizedMag,Unnamed: 11,Unnamed: 12,Unnamed: 13,Latitude_max_min,Longitude_max_min,Depth_max_min,Magnitude_max_min
0,23744.00,0.5724,19.246,145.616,Earthquake,131.6,6.0,0.590649,0.904493,0.189274,0.138889,NaN,NaN,min,-77.080,-179.997,-1.1,5.5
1,23746.00,0.4790,1.863,127.352,Earthquake,80.0,5.8,0.484060,0.853759,0.115675,0.083333,NaN,NaN,max,86.005,179.998,700.0,9.1
2,23747.00,0.7541,-20.579,-173.972,Earthquake,20.0,6.2,0.346451,0.016736,0.030096,0.194444,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,23750.00,0.7845,-59.076,-23.557,Earthquake,15.0,5.8,0.110396,0.434562,0.022964,0.083333,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,23751.00,0.5645,11.938,126.427,Earthquake,15.0,5.8,0.545838,0.851190,0.022964,0.083333,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
data.Type.value_counts()

Earthquake           23232
Nuclear Explosion      175
Explosion                4
Rock Burst               1
Name: Type, dtype: int64

In [5]:
data.isnull().sum()

Date                     0
Time                     0
Latitude                 0
Longitude                0
Type                     0
Depth                    0
Magnitude                0
NormalizedLat            0
NormalizedLong           0
NormalizedDepth          0
NormalizedMag            0
Unnamed: 11          23412
Unnamed: 12          23412
Unnamed: 13          23410
Latitude_max_min     23410
Longitude_max_min    23410
Depth_max_min        23410
Magnitude_max_min    23410
dtype: int64

In [6]:
col1 = data[["Latitude","Longitude", "Depth"]]
col2= data["Magnitude"]

In [7]:
col1.head()
col2.head()

0    6.0
1    5.8
2    6.2
3    5.8
4    5.8
Name: Magnitude, dtype: float64

In [8]:
InputX = col1.as_matrix()
Inputy = col2.as_matrix()

/home/hvyd/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/home/hvyd/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [9]:
print(Inputy)

[ 6.   5.8  6.2 ...,  5.9  6.3  5.5]


In [10]:
InputX.astype(float, copy = False)
Inputy.astype(float, copy = False)

array([ 6. ,  5.8,  6.2, ...,  5.9,  6.3,  5.5])

In [11]:
X_min = np.amin(InputX,0)
X_max = np.amax(InputX,0)
y_min = np.amin(Inputy,0)
y_max = np.amax(Inputy,0)
print("X_minimum", X_min)
print("X_maximum", X_max)
print("y_minimum", y_min)
print("y_maximum", y_max)

X_minimum [ -77.08  -179.997   -1.1  ]
X_maximum [  86.005  179.998  700.   ]
y_minimum 5.5
y_maximum 9.1


In [12]:
InputX_norm = (InputX-X_min)/ (X_max-X_min)
print(InputX_norm)

[[ 0.59064905  0.90449312  0.189274  ]
 [ 0.48406046  0.85375908  0.11567537]
 [ 0.34645124  0.01673634  0.03009556]
 ..., 
 [ 0.69900911  0.89007681  0.01583226]
 [ 0.41727749  0.82962513  0.11424904]
 [ 0.70194868  0.89281046  0.01859934]]


In [13]:
Inputy_norm = Inputy

In [14]:
X_feature = 3
y_feature = 1
sample = 23000 
InputX_reshape = np.resize(InputX_norm,(sample,X_feature))
Inputy_reshape = np.resize(Inputy_norm,(sample,y_feature))

In [15]:
print(InputX_reshape.shape)
print(Inputy_norm.shape)

(23000, 3)
(23412,)


In [16]:
batch_size = 20000
InputX_train = InputX_reshape[0:batch_size,:]
InputX_train.shape

(20000, 3)

In [17]:
Inputy_train = Inputy_reshape[0:batch_size,:]
Inputy_train.shape

(20000, 1)

In [18]:
v_size = 2500
InputX_test = InputX_reshape[batch_size: batch_size+v_size,:]
InputX_test.shape

(2500, 3)

In [19]:
Inputy_test = Inputy_reshape[batch_size: batch_size+v_size,:]
Inputy_test.shape

(2500, 1)

### Build and Train Model

In [20]:
learning_rate = 0.01
training_iterations = 10000
display_iterations = 20000


X = tf.placeholder(tf.float32,shape=(None,X_feature))
y = tf.placeholder(tf.float32, shape=(None,y_feature))

L1 = 3 
L2 = 3
L3 = 3
w_fc1 = tf.Variable(tf.random_uniform([X_feature,L1]))
b_fc1 = tf.Variable(tf.constant(0.1, shape=[3]))
w_fc2 = tf.Variable(tf.random_uniform([L1,L2]))
b_fc2 = tf.Variable(tf.constant(0.1, shape=[L2]))
w_fc3 = tf.Variable(tf.random_uniform([L2,L3]))
b_fc3 = tf.Variable(tf.constant(0.1, shape=[L3]))
W_fo = tf.Variable(tf.random_uniform([L3,y_feature]))
b_fo = tf.Variable(tf.constant(0.1,shape=[y_feature]))

matmul_fc1 = tf.matmul(X,w_fc1) + b_fc1
h_fc1 = tf.nn.relu(matmul_fc1) 
matmul_fc2 = tf.matmul(h_fc1,w_fc2) + b_fc2
h_fc2 = tf.nn.relu(matmul_fc2)
matmul_fc3 = tf.matmul(h_fc2,w_fc3) + b_fc3
h_fc3 = tf.nn.relu(matmul_fc3)
matmul_fc4 = tf.matmul(h_fc3,W_fo) + b_fo
output_layer = matmul_fc4

mean_sq = tf.reduce_mean(tf.square(y-output_layer))
train_step = tf.train.AdamOptimizer(learning_rate).minimize(mean_sq)


saver = tf.train.Saver()

In [21]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    print("Training_Loss:", sess.run([mean_sq], feed_dict={X:InputX_train,y:Inputy_train}))
    for i in range(training_iterations):
        sess.run([train_step], feed_dict={X:InputX_train,y:Inputy_train})
        if i%display_iterations ==0:
            print("Training_loss is :", sess.run(mean_sq, feed_dict={X:InputX_train,y:Inputy_train}),"at_iteration:", i)
            print("validation_loss is :", sess.run(mean_sq, feed_dict={X:InputX_test, y:Inputy_test}),"at_iteration:", i)
    
    save_path = saver.save(sess, "data/earthquake_model.ckpt")
    print("Model saved in file: %s" % save_path)
    print("final_training_loss:", sess.run(mean_sq, feed_dict={X:InputX_train, y:Inputy_train}))
    print("final_validation_loss", sess.run(mean_sq, feed_dict={X:InputX_test, y:Inputy_test}))

Training_Loss: [17.155544]
Training_loss is : 15.6167 at_iteration: 0
validation_loss is : 15.6785 at_iteration: 0
Model saved in file: data/earthquake_model.ckpt
final_training_loss: 0.176184
final_validation_loss 0.187261


### Make Prediction

In [22]:
lat = input("Enter latitude between -77 to 86 :")
long = input("Enter Longitude between -180 to 180:")
depth = input("Enter Depth between 0 to 700:")
InputX2 = np.asarray([lat, long, depth], dtype = np.float32)
InputX2_norm = (InputX2-X_min)/ (X_max-X_min)
InputX1_test = np.resize(InputX2_norm,(1,X_feature))
with tf.Session() as sess:
        saver.restore(sess, "/tmp/earthquake_model.ckpt")
        print("Model Restored.")
        print("output:", sess.run(output_layer, feed_dict={X:InputX1_test}), "Magnitude")

Enter latitude between -77 to 86 :39
Enter Longitude between -180 to 180:123
Enter Depth between 0 to 700:300
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
Model Restored.
output: [[ 3.06032753]] Magnitude
